In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# define basic parameters
year_span = [f'{i}_{i+2}' for i in range(1990,2020,3)]

##### Get the priliminary classifications

In [4]:
# prepare the different input band types
in_band_selection =['Spectrum',
                    'Spectrum_Normalize',
                    'Spectrum_Normalize_Fourier',
                    'Spectrum_Normalize_Fourier_Terrain',
                    'Spectrum_Normalize_Fourier_Terrain_Meterology']

In [10]:
# get the priliminary classifications and select those with a 
# >5 value in the 10-folds sum, here the 5 threshold comes from
# the Step_1 10-folds check of this fold

path = 'users/wangjinzhulala/North_China_Plain_Python/classification_img'

classification_gt_5 = {}

for in_band in in_band_selection:
    for year in year_span:
    
        preliminary_classification = ee.ImageCollection(f'{path}/{in_band}_{year}').sum()
        img_gt_5 = preliminary_classification.gt(5)
        
        classification_gt_5[(in_band,year)] = [img_gt_5]
        

In [15]:
# store the classification in a df for conviniency
classification_gt_5_df = pd.DataFrame(classification_gt_5)

# format the df
classification_checked = classification_gt_5_df.T.reset_index()
classification_checked.columns = ['In bands','year','classification']

##### Export the result

In [20]:
for idx,row in classification_checked.iterrows():
    
    out_path = 'users/wangjinzhulala/North_China_Plain_Python/classification_img_10_folds_checkd'
    region = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")
    
    # get data
    in_bands = row[0]
    year = row[1]
    img = row[2]
    
    # construct export name
    name = f'{in_bands}_{year}'
    
    # export
    task = ee.batch.Export.image.toCloudStorage(
                                                image = img,
                                                description=name,
                                                bucket='north_china_plain',
                                                fileNamePrefix=name,
                                                region=region.geometry().bounds(),
                                                scale=30,
                                                maxPixels=int(1e13),
                                                skipEmptyTiles=True)
    task.start()
    
    # print out the process
    print(f'{name} has been exported!')

Spectrum_1990_1992 has been exported!
Spectrum_1993_1995 has been exported!
Spectrum_1996_1998 has been exported!
Spectrum_1999_2001 has been exported!
Spectrum_2002_2004 has been exported!
Spectrum_2005_2007 has been exported!
Spectrum_2008_2010 has been exported!
Spectrum_2011_2013 has been exported!
Spectrum_2014_2016 has been exported!
Spectrum_2017_2019 has been exported!
Spectrum_Normalize_1990_1992 has been exported!
Spectrum_Normalize_1993_1995 has been exported!
Spectrum_Normalize_1996_1998 has been exported!
Spectrum_Normalize_1999_2001 has been exported!
Spectrum_Normalize_2002_2004 has been exported!
Spectrum_Normalize_2005_2007 has been exported!
Spectrum_Normalize_2008_2010 has been exported!
Spectrum_Normalize_2011_2013 has been exported!
Spectrum_Normalize_2014_2016 has been exported!
Spectrum_Normalize_2017_2019 has been exported!
Spectrum_Normalize_Fourier_1990_1992 has been exported!
Spectrum_Normalize_Fourier_1993_1995 has been exported!
Spectrum_Normalize_Fourier_1